In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import holidays as pyholidays

# read data from data folder (organized)

In [2]:
df=pd.read_excel('instance_organized.xlsx')
# convert string to list
df["DLI"]=df["DLI"].apply(lambda x: x.split(","))
df["DLI"]=df["DLI"].apply(lambda x: [int(i) for i in x])
# find position of 1's in DLI
df["DLI"]=df["DLI"].apply(lambda x: np.array([i for i, e in enumerate(x) if e == 1]))
df.rename(columns={"ID":"nurse_id",
                   "name":"nurse_name",
                   "vacations":"vacation_days",
                   "DLI":"dates_off",
                   "TF":"shift_preference",
                   "HA":"accumulated_hours"
                   },inplace=True)

dict_preference={0:"morning",1:"afternoon"}
df["shift_preference"]=df["shift_preference"].apply(lambda x: dict_preference[x])
df.reset_index(inplace=True,drop=True)
df.columns

Index(['nurse_id', 'nurse_name', 'vacation_days', 'shift_preference',
       'morning_availability_labor_day', 'morning_availability_weekend',
       'afternoon_availability_labor_day', 'afternoon_availability_weekend',
       'dates_off', 'accumulated_hours'],
      dtype='object')

In [3]:
df[['nurse_id', 'nurse_name',  'shift_preference',
       'morning_availability_labor_day', 'morning_availability_weekend',
       'afternoon_availability_labor_day', 'afternoon_availability_weekend',
       'dates_off', 'accumulated_hours']].to_json('nurses.json',orient='records')

In [4]:
shift_type={"morning":["M", "COM", "DISP"],"afternoon":["T1", "T2", "CHX"]}

demands={
    ("M",0,0):2,
    ("M",0,1):1,
    ("M",1,0):2,
    ("M",1,1):1,
    ("M",2,0):2,
    ("M",2,1):1,
    ("M",3,0):2,
    ("M",3,1):1,
    ("M",4,0):2,
    ("M",4,1):1,
    ("M",5,0):1,
    ("M",5,1):1,
    ("M",6,0):1,
    ("M",6,1):1,
    ("COM",0,0):1,
    ("COM",0,1):1,
    ("COM",1,0):1,
    ("COM",1,1):1,
    ("COM",2,0):1,
    ("COM",2,1):1,
    ("COM",3,0):1,
    ("COM",3,1):1,
    ("COM",4,0):1,
    ("COM",4,1):1,
    ("COM",5,0):0,
    ("COM",5,1):0,
    ("COM",6,0):0,
    ("COM",6,1):0,
    ("DISP",0,0):0,
    ("DISP",0,1):1,
    ("DISP",1,0):0,
    ("DISP",1,1):1,
    ("DISP",2,0):0,
    ("DISP",2,1):1,
    ("DISP",3,0):0,
    ("DISP",3,1):1,
    ("DISP",4,0):0,
    ("DISP",4,1):1,
    ("DISP",5,0):1,
    ("DISP",5,1):1,
    ("DISP",6,0):1,
    ("DISP",6,1):1,
    ("T1",0,0):1,
    ("T1",0,1):1,
    ("T1",1,0):1,
    ("T1",1,1):1,
    ("T1",2,0):1,
    ("T1",2,1):1,
    ("T1",3,0):1,
    ("T1",3,1):1,
    ("T1",4,0):1,
    ("T1",4,1):1,
    ("T1",5,0):0,
    ("T1",5,1):0,
    ("T1",6,0):0,
    ("T1",6,1):0,
    ("T2",0,0):0,
    ("T2",0,1):1,
    ("T2",1,0):0,
    ("T2",1,1):1,
    ("T2",2,0):0,
    ("T2",2,1):1,
    ("T2",3,0):0,
    ("T2",3,1):1,
    ("T2",4,0):0,
    ("T2",4,1):1,
    ("T2",5,0):1,
    ("T2",5,1):1,
    ("T2",6,0):1,
    ("T2",6,1):1,
    ("CHX",0,0):1,
    ("CHX",0,1):1,
    ("CHX",1,0):1,
    ("CHX",1,1):1,
    ("CHX",2,0):1,
    ("CHX",2,1):1,
    ("CHX",3,0):1,
    ("CHX",3,1):1,
    ("CHX",4,0):1,
    ("CHX",4,1):1,
    ("CHX",5,0):0,
    ("CHX",5,1):0,
    ("CHX",6,0):0,
    ("CHX",6,1):0
}


In [5]:
list_shifts=[]
# list of dates of january 2023
dates = pd.date_range(start='1/1/2023', end='1/31/2023')
shift_id=0
for fecha in dates:
    for shift in shift_type:
        for jornada in shift_type[shift]:
            list_shifts.append([shift_id, 
                                fecha.strftime("%Y-%m-%d"), 
                                fecha.week,
                                jornada, shift,fecha.weekday(),
                                fecha.strftime("%Y-%m-%d") in pyholidays.CO(years=2023),
                                demands[(jornada,fecha.weekday(),int( fecha.strftime("%Y-%m-%d") in pyholidays.CO(years=2023)))]
                                ])
            shift_id+=1
df_shifts=pd.DataFrame(list_shifts,columns=["shift_id","shift_date","week_of_year","shift","shift_type","weekday","holiday","demand"])
df_shifts.index=df_shifts["shift_id"]



In [6]:
df_shifts.to_json('shifts.json',orient='records',index=True)